In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_biweekly 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
AS
WITH
main AS (
    SELECT      
        a.business_unit,
        a.store_id,
        a.store_name,
        a.business_model,
        a.mien,
        a.city,
        a.mch2_id,
        a2.mch3_id,
        a2.mch4_id,
        a.product_id,
        a.product_name,
        a.manufacturer_id,
        a.manufacturer_name,
        a1.mdq,
        a1.external_report_mdq,
        a1.internal_report_mdq,
        a1.old_mdq,
        a1.mdq_age,
        a1.mdq_available,
        a.dc_id,
        a.dc_name,
        a.replenishment_mode,
        a.xd_dc_id,
        NULL AS dc_source,
        CASE WHEN a1.sku_type IS NULL THEN 'FRESH' ELSE a1.sku_type END AS sku_type,
        a.is_out,
        a.dc_sku_status,
        a.assortment_type
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_biweekly a1 USING (store_id, product_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a2 USING (product_id)
    UNION ALL 
    SELECT      
        a.business_unit,
        a.store_id,
        a.store_name,
        CASE
            WHEN a.business_unit = '1500' THEN 'Supermarket'
            WHEN a.business_unit = '2000' AND a.concept IN ('WiN','Rural') THEN a.concept
            ELSE 'Urban'
        END business_model,
        a.mien,
        a.city,
        a2.mch2_id,
        a2.mch3_id,
        a2.mch4_id,
        a2.product_id,
        a2.product_name,
        a2.manufacturer_id,
        a2.manufacturer_name,
        a1.mdq,
        a1.external_report_mdq,
        a1.internal_report_mdq,
        a1.old_mdq,
        a1.mdq_age,
        a1.mdq_available,   
        NULL AS dc_id,
        NULL AS dc_name,
        NULL AS replenishment_mode,
        NULL AS xd_dc_id,
        NULL AS dc_source,
        a1.sku_type,
        NULL is_out,
        NULL dc_sku_status,
        NULL assortment_type
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_biweekly a1 
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store a USING (store_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a2 USING (product_id)
    WHERE a1.mdq IS NOT NULL 
        AND a1.sku_type = 'FRESH'
)
SELECT      
    a.*,
    IF(a1.sku IS NOT NULL, 'YES','NO') is_problem_article
FROM main a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store a2 USING (store_id)
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_problem_article a1 ON a.business_unit = a1.business_unit 
    AND a2.region_vn = a1.area
    AND a.product_id = a1.sku

""")

In [0]:
spark.sql(f"""
          DELETE FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_biweekly_his WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
          """)

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_biweekly_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    business_unit,
    store_id,
    business_model,
    mch2_id,
    product_id,
    manufacturer_id,
    mdq,
    external_report_mdq,
    internal_report_mdq,
    old_mdq,
    mdq_age,
    mdq_available,
    dc_id,
    replenishment_mode,
    xd_dc_id,
    sku_type,
    is_out,
    dc_sku_status,
    assortment_type,
    is_problem_article
FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_biweekly; 
""")